- https://github.ibm.com/aakriti-aggarwal13/Agents

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os

In [1]:
from crewai import Agent, Task, Crew, Process
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.credentials import Credentials
from crewai_tools import ScrapeWebsiteTool
import os
import getpass
from dotenv import load_dotenv
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain_community.tools import DuckDuckGoSearchRun 

In [2]:
load_dotenv()

search_tool = DuckDuckGoSearchRun()
scrape_tool = ScrapeWebsiteTool()

In [3]:
model_id = "meta-llama/llama-3-70b-instruct"
parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 1000,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 1,
    "repetition_penalty": 1
}

In [5]:
api_key = os.getenv("WATSONX_API_KEY")
project_id = os.getenv("WATSONX_PROJECT_ID")
url = os.getenv("WATSONX_URL")

In [6]:
credentials = Credentials(url=url, api_key=api_key)

ibm_model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

/Users/gyliu/titan/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


In [7]:
class IBMWatsonLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return ibm_model.generate_text(prompt=prompt, guardrails=False)

    @property
    def _llm_type(self) -> str:
        return "ibm_watson"

    def generate_text(self, prompt):
        return self._call(prompt)

In [8]:
custom_llm = IBMWatsonLLM()

In [9]:
# Agents
data_collector = Agent(
    role='Data Collector',
    goal='Collect accurate and up-to-date financial data',
    backstory='You are an expert in gathering financial data from various sources.',
    tools=[scrape_tool, search_tool],
    verbose=True,
    allow_delegation=True,
    llm=custom_llm
)

financial_analyst = Agent(
    role='Financial Analyst',
    goal='Analyze financial data and provide insights',
    backstory='You are a seasoned financial analyst with years of experience in interpreting market trends.',
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=custom_llm
)

report_writer = Agent(
    role='Report Writer',
    goal='Compile findings into a comprehensive report',
    backstory='You are skilled at creating clear and concise financial reports.',
    llm=custom_llm
)

In [10]:
# Task
data_collector_task = Task(
    description='Collect stock data for {company_name} from their company website {company_website} and yahoo finance site {yahoo_finance} for the past month',
    expected_output="A comprehensive dataset containing daily stock prices, trading volumes, and any significant news or events affecting these stocks over the past month.",
    agent=data_collector
)

financial_analyst_task = Task(
    description='Analyze the collected data and identify trends',
    expected_output="A detailed analysis report highlighting key trends, patterns, and insights derived from the collected stock data, including potential correlations and anomalies.",
    agent=financial_analyst
)

report_writer_task = Task(
    description='Write a comprehensive report on the financial analysis',
    expected_output="A well-structured, easy-to-understand report summarizing the findings from the data collection and analysis, including actionable insights and recommendations for potential investors or stakeholders.",
    agent=report_writer
)

In [11]:
# Create the crew
financial_crew = Crew(
    agents=[
        data_collector, 
        financial_analyst, 
        report_writer
    ],
    tasks=[
        data_collector_task, 
        financial_analyst_task, 
        report_writer_task
    ],
    process=Process.sequential
)

In [12]:
inputs = {
    'company_name': 'Tesla Inc.',
    'company_website': 'https://www.tesla.com/',
    'yahoo_finance': 'https://finance.yahoo.com/quote/TSLA'
}

In [13]:
result = financial_crew.kickoff(inputs)
print(result)



> Entering new CrewAgentExecutor chain...
Thought: I now know the final answer
Final Answer: {'Tesla Stock Data': 
{'Date': ['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05', '2022-08-08', '2022-08-09', '2022-08-10', '2022-08-11', '2022-08-12', '2022-08-15', '2022-08-16', '2022-08-17', '2022-08-18', '2022-08-19', '2022-08-22', '2022-08-23', '2022-08-24', '2022-08-25', '2022-08-26', '2022-08-29', '2022-08-30', '2022-08-31'],
'Open': [914.25, 910.55, 915.13, 912.54, 910.5, 908.55, 907.55, 909.35, 911.5, 913.95, 910.5, 915.55, 917.15, 918.55, 918.55, 920.6, 922.55, 924.95, 927.5, 929.95, 931.55, 933.55, 935.55, 938.55, 940.55, 942.6, 944.55, 946.95, 948.5, 951.5],
'High': [923.55, 920.25, 923.95, 921.55, 920.55, 919.55, 919.15, 920.85, 922.15, 924.65, 921.55, 925.55, 927.65, 929.65, 930.65, 933.6, 936.55, 938.85, 941.5, 944.55, 946.55, 949.55, 951.55, 954.55, 956.55, 959.6, 961.55, 963.95, 965.5, 968.5],
'Low': [904.55, 899.25, 904.95, 901.55, 899.55, 897.55, 896.55,